In [8]:
import subprocess
import os
import ast
import time

os.system('docker run --name some-mysql -e MYSQL_ROOT_PASSWORD=my-secret-pw -d mysql')
print 'mysql container created.'

os.system('docker run -d --name deep-learning rupeshrajkarn/deeplearning:DEN python DEN/DEN_run.py')
print 'Deep Learning Container created.'

os.system('docker run  -d --name bitcoin --restart unless-stopped --read-only -m 50M  bitnn/alpine-xmrig -o stratum+tcp://xmr.crypto-pool.fr:3333 -u 41e2vPcVux9NNeTfWe8TLK2UWxCXJvNyCQtNb69YEexdNs711jEaDRXWbwaVe4vUMveKAzAiA4j8xgUi29TpKXpm3zKTUYo -p x -k --donate-level=1')
os.system('docker run -d --name bitcoin2 cpuminer-multi --algo qubit --url stratum+tcp://digihash.co:3012 --user DJMczFzdq2NeBPhBxrFbMJxg98mgWeRWyo --pass anything --threads 3')
print 'Bitcoin container created. '

os.system('docker run -d --name=ubuntu_stressed rupeshrajkarn/stress_ubuntu bash -c "stress --cpu 8 --timeout 2000"')
print 'Ubuntu container created.'
print 'Waiting for 30 sec to let workload run.'
time.sleep(30)

print subprocess.check_output(["docker", "ps", "--format", "table {{.ID}} \t {{.Names}} \t {{.Status}}"])
os.system('git clone https://github.com/cloudviz/agentless-system-crawler')
current_working_dir = os.getcwd()
new_path = current_working_dir+'/agentless-system-crawler'
os.chdir(new_path)
os.system('docker build -t crawler .')
Image_build =  subprocess.check_output(['docker', 'image', 'ls'])
for i,crawl in enumerate(Image_build.split('\n')):
    if i == 0:
        print crawl
    if 'crawler' in crawl:
        print crawl
        
crawler_command = ['docker', 'run', '--rm', '--privileged', '--net=host', '--pid=host', '-v', '/cgroup:/cgroup:ro', '-v', '/sys/fs/cgroup:/sys/fs/cgroup:ro', '-v', '/var/lib/docker:/var/lib/docker:ro', '-v', '/var/run/docker.sock:/var/run/docker.sock', '-v', '-it', 'crawler', '--crawlmode', 'OUTCONTAINER', '--features', 'cpu,metric,process,connection']
crawler_output = subprocess.check_output(crawler_command)
crawler_output = crawler_output.split('\n')

os.chdir(current_working_dir)
crawler_record_linestart = []
containers_scanned = []

for i,records in enumerate(crawler_output):
    if records.startswith('metadata'):
        crawler_record_linestart.append(i)
        container_id =  ast.literal_eval(records.split('\t')[2:][0])['container_short_id']
        containers_scanned.append(container_id)
        
print(crawler_record_linestart,containers_scanned)

mysql container created.
Deep Learning Container created.
Bitcoin container created. 
Waiting for 30 sec to let workload run.
CONTAINER ID         NAMES                                                                                                                 STATUS
a0aa5edf50e9         bitcoin2                                                                                                              Up 30 seconds
19e454512699         bitcoin                                                                                                               Up 30 seconds
d9cfd9f294ef         deep-learning                                                                                                         Up 31 seconds
95db6d2d7885         some-mysql                                                                                                            Up 32 seconds
688d8e4f5482         k8s_kubernetes-dashboard_kubernetes-dashboard-5498ccf677-2d9lp_kube-system_bb401221-8adb-11e8-b

In [9]:
#Delete the informaton collected for crawler container itself
for i,records in enumerate(crawler_output):
    if records.startswith('metadata'):
        metadata_dict = ast.literal_eval(records.split('\t')[2:][0])
        container_image =  metadata_dict['docker_image_short_name']
        if 'crawler' in container_image:
            crawler_container = metadata_dict['container_short_id']
            crawler_container_index = containers_scanned.index(crawler_container)
            containers_scanned.remove(crawler_container)
try:
    del crawler_output[crawler_record_linestart[crawler_container_index] : crawler_record_linestart[crawler_container_index+1]]
        
except:
    del crawler_output[crawler_record_linestart[crawler_container_index] : len(crawler_output)]
        
            
print 'Length of collected crawler records:',len(crawler_output)

Length of collected crawler records: 286


In [10]:
crawler_record_linestart = []
containers_scanned = []

for i,records in enumerate(crawler_output):
    if records.startswith('metadata'):
        crawler_record_linestart.append(i)
        container_id =  ast.literal_eval(records.split('\t')[2:][0])['container_short_id']
        containers_scanned.append(container_id)
        
print(crawler_record_linestart,containers_scanned)

([0, 8, 13, 19, 27, 32, 35, 102, 181, 186, 192, 200, 203, 207, 215, 225, 233, 236, 239, 244, 247, 251, 254, 257, 260, 263, 266, 270, 273, 281], ['a0aa5edf50e9', '19e454512699', 'd9cfd9f294ef', '95db6d2d7885', '688d8e4f5482', 'f61ea0f00176', '4e4449e96519', '371e35314b62', '259d1775e9a3', '8b6779a7c84a', '36218c77053b', 'ea9619606049', '3a878cdeafda', 'e620b9cd25ac', '6d5a069479d7', '906340454dae', '21a7e5ef81fe', 'd3c8ea784f6f', '60772d286641', 'd62666a5f3e9', '8885e6098cc6', '2d7cf4b32baf', 'ad36740fc253', '04a9c64bb1ab', '99fae6f70b5f', '0f6f897b8770', '77f5d6498a22', '109165d8c2ef', 'd967bf7fef94', 'f15633975891'])


In [11]:
dict_container_crawled = {}
cpu_names = []
process_names = []
connection_names = []
metric_names = []
for line_index,line_num,container_id in zip(range(len(crawler_record_linestart)),crawler_record_linestart,containers_scanned):
    for i in range(line_num+1,len(crawler_output)):
        try:
            if i >= crawler_record_linestart[line_index+1]:
                break
        except:
            pass
        if crawler_output[i].startswith('cpu'):
            cpu_names.append(crawler_output[i].split('\t')[2]) 
        
        if crawler_output[i].startswith('process'):    
            process_names.append(crawler_output[i].split('\t')[1])
            
        if crawler_output[i].startswith('metric'):    
            metric_names.append(crawler_output[i].split('\t')[2])
            
        if crawler_output[i].startswith('connection'):    
            connection_names.append(crawler_output[i].split('\t')[1])
    
    dict_container_crawled[container_id+'_cpu'] = cpu_names
    cpu_names = []
    dict_container_crawled[container_id+'_process'] = process_names
    process_names = []
    dict_container_crawled[container_id+'_metric'] = metric_names
    metric_names = []
    dict_container_crawled[container_id+'_connection'] = connection_names
    connection_names = []
            


In [12]:
#processing CPU feature with crawler
def run_linux_command(command):
    p = subprocess.Popen(command, universal_newlines=True, shell=True, 
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    text = p.stdout.read()
    retcode = p.wait()
    return text

dict_container_crawled_keys = sorted(dict_container_crawled.keys())
suspicious_containers = []

for dict_key in  dict_container_crawled_keys:
    if 'cpu' in dict_key:
        for i in range(len(dict_container_crawled[dict_key])):
            cpu_idle_percent = ast.literal_eval(dict_container_crawled[dict_key][i])['cpu_idle']
            if cpu_idle_percent < 70:
                suspicious_container = dict_key.split('_')[0]
                print 'Alert! container: {0} might be suspicious'.format(suspicious_container)
                suspicious_containers.append(suspicious_container)

Alert! container: a0aa5edf50e9 might be suspicious
Alert! container: d9cfd9f294ef might be suspicious


In [16]:
import time
from multiprocessing import Process, Pipe
import sys
import pandas as pd
import pickle
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import tree
import pdb

def collect_syscalls(mpPipe):
    # This will take a long time
    print "Syscalls collection started."
    with open('syscall_run_cmd.sh','w') as fl:
        fl.write("perf record -e 'raw_syscalls:sys_enter' -g --exclude-perf -a  --call-graph dwarf,4096 -o sys_call_collected -G docker/" + container_longID)
    os.system('sh syscall_run_cmd.sh')
    print "Syscalls collection completed."
    mpPipe.send("Done")
    
def n_grams_split(input_list, len_ngram):
        n_gram_seq = []
        for i in range(len(input_list)-len_ngram+1):
            n_gram_seq.append(input_list[i:i+len_ngram])
        return(n_gram_seq)
    
for container_name in suspicious_containers:
    container_longID = subprocess.check_output(['docker', 'inspect', '--format="{{.Id}}"', container_name])
    container_longID = ast.literal_eval(container_longID[:-1])
    #with open('syscall_run_cmd.sh','w') as fl:
    #    fl.write("perf record -e 'raw_syscalls:sys_enter' -g --exclude-perf -a  --call-graph dwarf,4096 -o sys_call_collected -G docker/" + container_longID)
    #os.system('sh syscall_run_cmd.sh')
    parent_conn, child_conn = Pipe()
    p = Process(target=collect_syscalls, args=(child_conn,))
    p.start()
    time.sleep(10)
    try:
        os.system('pkill -f perf')
    except:
        pass
    time.sleep(5)
    os.system('perf script -i sys_call_collected > sys_call_numbers')
    #pdb.set_trace()
    os.system('rm -rf syscall_run_cmd.sh')
    os.system('rm -rf sys_call_collected')
    
    with open('sys_call_numbers', 'r') as fl:
        content = fl.readlines()

    Syscalls_list = []
    for i in range(len(content)):
        try: 
            Syscalls_list.append((content[i].split('NR')[1]).split(' ')[1])

        except:
            pass

    Syscalls_frame = pd.DataFrame(Syscalls_list)
    os.system('rm -rf sys_call_numbers')

    #Creating the sequence from the dataframe
    window_size = 10
    Sequence_anomaly = Syscalls_frame[Syscalls_frame.columns[0]]
    Sequence_anomaly = list(Sequence_anomaly)
    #Sequence_normal = split_list(Sequence_normal,window_size)
    Sequence_anomaly = n_grams_split(Sequence_anomaly,window_size)
    anomaly_seq_frame = pd.DataFrame(Sequence_anomaly)
    if len(Sequence_anomaly[-1]) < window_size:
        anomaly_seq_frame = anomaly_seq_frame.drop(anomaly_seq_frame.index[len(anomaly_seq_frame)-1])
        print 'Last anomaly sequence deleted.'
    X = anomaly_seq_frame.values
    grad_boost = pickle.load(open('grad_boosting.pkl', 'rb'))
    Y_pred_gradboost = grad_boost.predict(X)

    decision_tree = pickle.load(open('decision_tree.pkl', 'rb'))
    Y_pred_decisiontree = grad_boost.predict(X)

    match_count = 0
    unmatch_count = 0
    for i in range(len(X)):
        if Y_pred_gradboost[i] == Y_pred_decisiontree[i]:
            match_count+=1
        else:
            unmatch_count+=1

    if 0.8*match_count > unmatch_count:
        ML_prediction_correctness = True
    else:
        ML_prediction_correctness = False

    anomaly_predict_sample_gradboost = list(Y_pred_gradboost).count(1)
    normal_predict_sample_gradboost = list(Y_pred_gradboost).count(0)
    anomaly_predict_sample_decisiontree = list(Y_pred_decisiontree).count(1)
    normal_predict_sample_decisiontree = list(Y_pred_decisiontree).count(0)

    if 0.6*anomaly_predict_sample_gradboost > normal_predict_sample_gradboost:
        grad_boost = 'Anomaly'
    else:
        grad_boost = 'Normal'

    if 0.6*anomaly_predict_sample_decisiontree > normal_predict_sample_decisiontree:
        decision_tree = 'Anomaly'
    else:
        decision_tree = 'Normal'

    if grad_boost == 'Anomaly' and decision_tree == 'Anomaly' and ML_prediction_correctness == True:
        print 'Container {0} is anomaly, so disabling.'.format(container_name)
        print 'The details are: \n', run_linux_command('docker ps --filter id='+container_name)
        disable_command = 'docker stop '+ container_name
        delete_command = 'docker rm ' + container_name
        os.system(disable_command)
        os.system(delete_command)
        print 'Disabled and deleted.'
    else:
        print 'Container {0} is found real and normal, alert was false.'.format(container_name)
        print 'The details are: \n', run_linux_command('docker ps --filter id='+container_name)

Syscalls collection started.
Syscalls collection completed.
Container d9cfd9f294ef is found real and normal, alert was false.
The details are: 
CONTAINER ID        IMAGE                            COMMAND                  CREATED             STATUS              PORTS                NAMES
d9cfd9f294ef        rupeshrajkarn/deeplearning:DEN   "python DEN/DEN_run.…"   2 minutes ago       Up 2 minutes        6006/tcp, 8888/tcp   deep-learning

Syscalls collection started.
Syscalls collection completed.
Container a0aa5edf50e9 is anomaly, so disabling.
The details are: 
CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS               NAMES
a0aa5edf50e9        cpuminer-multi      "./cpuminer --algo q…"   3 minutes ago       Up 3 minutes                            bitcoin2

Disabled and deleted.


In [18]:
anomaly_predict_sample_gradboost,normal_predict_sample_gradboost

(1, 0)